<a href="https://colab.research.google.com/github/neeraj-iitm/Recommender-Systems-Capstone/blob/main/01_Non_Personalised_RS_assignment/02_Product%20Association_RS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
import pandas as pd
import numpy as np

In [28]:
url_m = "https://raw.githubusercontent.com/neeraj-iitm/Recommender-Systems-Capstone/main/01_Non_Personalised_RS_assignment/data/movies.csv"
url_r = "https://raw.githubusercontent.com/neeraj-iitm/Recommender-Systems-Capstone/main/01_Non_Personalised_RS_assignment/data/ratings.csv"
url_t = "https://raw.githubusercontent.com/neeraj-iitm/Recommender-Systems-Capstone/main/01_Non_Personalised_RS_assignment/data/tags.csv"

In [29]:
movies   = pd.read_csv(url_m,encoding = "ISO-8859-1")
ratings  = pd.read_csv(url_r)
tags     = pd.read_csv(url_t,encoding = "ISO-8859-1")

In [30]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [34]:
movies.title= movies.title.apply(lambda x: x.split(" (")[0])

In [36]:
movies.genres = movies.genres.apply(lambda x : x.replace("|", ","))

In [37]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(stop_words = 'english')

In [38]:
mtx = tfidf.fit_transform(movies.genres)

In [39]:
mtx.shape

(2500, 21)

In [40]:
from sklearn.metrics.pairwise import linear_kernel

cosine_sim = linear_kernel(mtx, mtx)

In [41]:
cosine_sim

array([[1.        , 0.78157913, 0.16691005, ..., 0.        , 0.        ,
        0.18548916],
       [0.78157913, 1.        , 0.        , ..., 0.        , 0.        ,
        0.23732614],
       [0.16691005, 0.        , 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.        ,
        0.63001119],
       [0.        , 0.        , 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.18548916, 0.23732614, 0.        , ..., 0.63001119, 0.        ,
        1.        ]])

In [42]:
indices = pd.Series(movies.index, index=movies['title']).drop_duplicates()

In [69]:
indices

title
Toy Story                         0
Jumanji                           1
Grumpier Old Men                  2
Waiting to Exhale                 3
Father of the Bride Part II       4
                               ... 
Interstellar                   2495
X-Men: Days of Future Past     2496
Edge of Tomorrow               2497
Gone Girl                      2498
Guardians of the Galaxy        2499
Length: 2500, dtype: int64

In [ ]:
title = "Star Wars: Episode IV - A New Hope"
idx = indices[title]

sim_scores = list(enumerate(cosine_sim[idx]))

In [73]:
sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

In [101]:
sim_scores[30:40]

[(2367, 1.0000000000000002),
 (2496, 1.0000000000000002),
 (2499, 1.0000000000000002),
 (847, 0.953335982160903),
 (1801, 0.953335982160903),
 (775, 0.9431384373467113),
 (1641, 0.9431384373467113),
 (1724, 0.9431384373467113),
 (2371, 0.9431384373467113),
 (276, 0.9208587534044265)]

In [77]:
movie_indices = [i[0] for i in sim_scores[1:11]]

In [78]:
movie_indices

[157, 187, 261, 584, 597, 720, 872, 994, 997, 1193]

In [43]:
def content_recommender(title, cosine_sim=cosine_sim, df=movies, indices=indices):
    # Obtain the index of the movie that matches the title
    idx = indices[title]

    # Get the pairwsie similarity scores of all movies with that movie
    # And convert it into a list of tuples as described above
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the cosine similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movies. Ignore the first movie.
    sim_scores = sim_scores[1:11]

    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    return df['title'].iloc[movie_indices]


In [51]:
content_recommender('Star Wars: Episode IV - A New Hope')

157                 Star Wars: Episode IV - A New Hope
187                                           Stargate
261                                     Demolition Man
584     Star Wars: Episode V - The Empire Strikes Back
597         Star Wars: Episode VI - Return of the Jedi
720                Star Trek III: The Search for Spock
872                                      Lost in Space
994                                     Rocketeer, The
997                                               Tron
1193                                       Logan's Run
Name: title, dtype: object

In [130]:
grouped_tags = tags.groupby("movieId")["tag"].apply(",".join)

In [134]:
grouped_tags

movieId
1         children,computer animation,Disney,family,Pixa...
2         fantasy,Robin Williams,scary,time travel,Child...
3         comedinha de velhinhos engraÃ§ada,Jack Lemmon,...
4            characters,chick flick,CLV,chick flick,revenge
5         steve martin,pregnancy,remake,Fantasy,CLV,Dian...
                                ...                        
109487    bad science,cliché finale,relativity,sci-fi,so...
111362    time travel,Evan Peters,Fan Bingbing,Hugh Jack...
111759    based on a novel,Reluctant Hero,time travel,Em...
112556    David Fincher,good adaptation,meticulous,mindf...
112852    action,characters,chris pratt,fun,music,ost,st...
Name: tag, Length: 2495, dtype: object